### Sample notebook for prediction against image files  

#### Import libraries  

In [1]:
import os,glob
from PIL import Image
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

2021-09-24 10:22:35.107115: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-24 10:22:35.107191: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


#### Parameters  

In [2]:
w=56
work_dir="data/dogs_cats"
image_dir=work_dir+"/original/unknown"
pred_dir=work_dir+"/predicted"
results_base='dogs_cats-results'
model_file="dogs_cats-model-w{0}.h5".format(w)
classes=["dogs","cats"]
num_classes=len(classes)

#### Read model file  

In [3]:
model=load_model(model_file)

2021-09-24 10:22:38.805307: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-09-24 10:22:38.805364: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-24 10:22:38.805391: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (anhvu): /proc/driver/nvidia/version does not exist
2021-09-24 10:22:38.806348: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### Prediction against an image file one by one  

In [4]:
# Open file for saving results  
results_file='{0}-w{1}.csv'.format(results_base,w)
res=open(results_file,'w')
res.write("file,{0},{1}\n".format(classes[0],classes[1]))

# Remove files in output directory
files=glob.glob(pred_dir+"/*.*")
for f in files:
    os.remove(f)

# Do prediction 
files=glob.glob(image_dir+"/*.*")
for f in files:
    # Read image file
    img=Image.open(f)
    img=img.convert("RGB")
    img=img.resize((w,w))
    # Reshape into 4-d ndarray
    X=np.array(img).reshape(1,w,w,3).astype("float16")
    X_pred=X/255
    # Classification prediction
    pred=model.predict(X_pred)
    # Output image into pred_dir
    pred_ans=pred.argmax()
    pred_cls=classes[pred_ans]
    tag="pred_as_{0}-w{1}".format(pred_cls,w)
    base=os.path.basename(f)
    img_file="{0}/{1}.{2}.jpg".format(pred_dir,os.path.splitext(base)[0],tag)
    img.save(img_file)
    print(img_file,pred)
    res.write("{0},{1},{2}\n".format(f,pred[0][0],pred[0][1]))

# Close results file
res.close()

print("OK")

2021-09-24 10:22:39.711458: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


data/dogs_cats/predicted/9.pred_as_dogs-w56.jpg [[0.5331384 0.4668616]]
data/dogs_cats/predicted/2.pred_as_dogs-w56.jpg [[0.65463257 0.3453674 ]]
data/dogs_cats/predicted/4.pred_as_dogs-w56.jpg [[0.99724966 0.00275027]]
data/dogs_cats/predicted/3.pred_as_dogs-w56.jpg [[0.6427231  0.35727695]]
data/dogs_cats/predicted/6.pred_as_dogs-w56.jpg [[0.6063182  0.39368182]]
data/dogs_cats/predicted/8.pred_as_cats-w56.jpg [[0.4622681 0.5377319]]
data/dogs_cats/predicted/1.pred_as_dogs-w56.jpg [[0.57824546 0.4217545 ]]
data/dogs_cats/predicted/7.pred_as_cats-w56.jpg [[0.4275902 0.5724098]]
data/dogs_cats/predicted/5.pred_as_dogs-w56.jpg [[0.5870313 0.4129687]]
data/dogs_cats/predicted/0.pred_as_dogs-w56.jpg [[0.731671 0.268329]]
OK
